In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
import glob, os, sys
import pickle

In [2]:
from keras import backend as K
from keras.preprocessing import image
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import optimizers

from sklearn.model_selection import train_test_split

from keras.applications.resnet50 import ResNet50 as resnet
from keras.applications.resnet50 import preprocess_input as resnet_pp

from keras.optimizers import Adam

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
main_path = os.path.abspath("d://data/POETdataset/")
classes = ['aeroplane', 'boat', 'dog', 'bicycle', 'cat', 'cow', 'diningtable', 'horse', 'motorbike','sofa']
IMG_SIZE = 224

In [4]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
def load_object(filename):
    with open(filename, 'rb') as fp:
        return pickle.load(fp)

In [5]:
# Build a model by adding preprocessing before the pretrained CNN
def get_feature_extraction_model():
    cnn_object, pp_function, img_size = _get_pretrained_model()
    model = keras.models.Sequential()
    cnn_model = cnn_object(weights='imagenet', include_top=False, pooling='max')
    model.add(keras.layers.Lambda(pp_function, name='preprocessing', input_shape=(img_size, img_size, 3)))
    model.add(cnn_model)
    return model

# Unpacking information from the models dictionary
def _get_pretrained_model():
    cnn_object = resnet
    pp_function = resnet_pp
    img_size = IMG_SIZE
    return cnn_object, pp_function, img_size

def get_features(files, model):
    # Load images based on the size of the Lambda layer 
    # provided as the first layer before the pretrained CNN
    x = np.array([image.img_to_array(image.load_img(file, target_size=(model.layers[0].input_shape[1], model.layers[0].input_shape[1]))) for file in files])
    return model.predict(x, verbose=1)

In [6]:
model = get_feature_extraction_model()

In [7]:
files_list = [glob.glob(os.path.join(main_path, 'PascalImages', '%s*' %class_)) for class_ in classes]

for files in files_list:
    assert len(files) > 0
    
new_dir = 'features'
if not os.path.exists(new_dir):
    os.makedirs(new_dir)

files_dict = {class_name.replace('*', ''): class_files for class_name, class_files in zip(classes, files_list)}
for class_number, (class_name, files) in enumerate(files_dict.items()):
    print(class_name)
    features = get_features(files, model)
    print(features.shape)
    save_object(features, new_dir + os.path.sep + class_name+'_features.pkl')

aeroplane
666/666 [==============================] - 9s 14ms/step
(666, 2048)
boat
504/504 [==============================] - 5s 9ms/step
(504, 2048)
dog
1257/1257 [==============================] - 11s 9ms/step
(1257, 2048)
bicycle
536/536 [==============================] - 5s 9ms/step
(536, 2048)
cat
1051/1051 [==============================] - 9s 9ms/step
(1051, 2048)
cow
301/301 [==============================] - 3s 10ms/step
(301, 2048)
diningtable
498/498 [==============================] - 5s 9ms/step
(498, 2048)
horse
480/480 [==============================] - 4s 9ms/step
(480, 2048)
motorbike
510/510 [==============================] - 5s 10ms/step
(510, 2048)
sofa
467/467 [==============================] - 4s 9ms/step
(467, 2048)


In [15]:
X = []
y = []
for class_number, (class_name, files) in enumerate(files_dict.items()):
    classfeatures_path = os.path.abspath(os.path.join('.', 'features', class_name + '_features.pkl'))
    print(classfeatures_path)
    features = load_object(classfeatures_path).reshape(-1,2048)
    print(features.shape)
    X.append(features)
    y.append([class_number] * features.shape[0])
    
X = np.concatenate(X)
y = np.concatenate(y)
y = keras.utils.to_categorical(y)
X.shape, y.shape

D:\data\cogsci3\features\aeroplane_features.pkl
(666, 2048)
D:\data\cogsci3\features\boat_features.pkl
(504, 2048)
D:\data\cogsci3\features\dog_features.pkl
(1257, 2048)
D:\data\cogsci3\features\bicycle_features.pkl
(536, 2048)
D:\data\cogsci3\features\cat_features.pkl
(1051, 2048)
D:\data\cogsci3\features\cow_features.pkl
(301, 2048)
D:\data\cogsci3\features\diningtable_features.pkl
(498, 2048)
D:\data\cogsci3\features\horse_features.pkl
(480, 2048)
D:\data\cogsci3\features\motorbike_features.pkl
(510, 2048)
D:\data\cogsci3\features\sofa_features.pkl
(467, 2048)


((6270, 2048), (6270, 10))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5643, 2048), (5643, 10), (627, 2048), (627, 10))

In [17]:
dense_model = Sequential()
# model.add(Flatten(input_shape=(1,1,2048)))
dense_model.add(Dense(units=128, activation='relu', input_shape=(2048,)))
dense_model.add(Dropout(0.25))
dense_model.add(Dense(units=10, activation='softmax'))

dense_model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy'])
history = dense_model.fit(X_train,y_train,
                    batch_size = 32,
                    epochs = 100,
                    validation_data=(X_test, y_test))

Train on 5643 samples, validate on 627 samples
Epoch 1/100
5643/5643 [==============================] - 2s 332us/step - loss: 5.9879 - acc: 0.4916 - val_loss: 0.9175 - val_acc: 0.6794
Epoch 2/100
5643/5643 [==============================] - 1s 123us/step - loss: 0.9088 - acc: 0.7163 - val_loss: 0.7247 - val_acc: 0.8038
Epoch 3/100
5643/5643 [==============================] - 1s 123us/step - loss: 0.7188 - acc: 0.7618 - val_loss: 0.7199 - val_acc: 0.7831
Epoch 4/100
5643/5643 [==============================] - 1s 124us/step - loss: 0.6217 - acc: 0.7948 - val_loss: 0.6747 - val_acc: 0.8214
Epoch 5/100
5643/5643 [==============================] - 1s 125us/step - loss: 0.5866 - acc: 0.8005 - val_loss: 0.6801 - val_acc: 0.8166
Epoch 6/100
5643/5643 [==============================] - 1s 126us/step - loss: 0.5295 - acc: 0.8182 - val_loss: 0.6621 - val_acc: 0.8293
Epoch 7/100
5643/5643 [==============================] - 1s 123us/step - loss: 0.4930 - acc: 0.8347 - val_loss: 0.6819 - val_acc: 0

KeyboardInterrupt: 